In [7]:
import config
#DB instance creation
import mysql.connector
from mysql.connector import errorcode
import requests


In [8]:
def connect_to_db():
    cnx = mysql.connector.connect(
    host=config.host, user=config.user, passwd=config.password, database=config.DB_NAME)
    cursor = cnx.cursor()
    return cnx, cursor


In [17]:
#helper method, executes a sql query and returns the fetchall results
def execute_query(query):
    cnx, c = connect_to_db()
    c.execute(query)
    result = c.fetchall()
    #close
    c.close()
    cnx.close()
    return result

In [10]:
def get_genres_from_db():
    cnx, cursor = connect_to_db()
    query = """SELECT * 
                FROM genres"""
    cursor.execute(query)
    return cursor.fetchall()

get_genres_from_db()

[('10402', 'Music'),
 ('10749', 'Romance'),
 ('10751', 'Family'),
 ('10752', 'War'),
 ('10770', 'TV Movie'),
 ('12', 'Adventure'),
 ('14', 'Fantasy'),
 ('16', 'Animation'),
 ('18', 'Drama'),
 ('27', 'Horror'),
 ('28', 'Action'),
 ('35', 'Comedy'),
 ('36', 'History'),
 ('37', 'Western'),
 ('53', 'Thriller'),
 ('80', 'Crime'),
 ('878', 'Science Fiction'),
 ('9648', 'Mystery'),
 ('99', 'Documentary')]

In [24]:

def convert_list_from_tuples(tuple_list):
    """The movie titles are retrieved as a list of tupes, 
    return instead a list with just the title from the first half of each tuple"""
    title_list = []
    for item in tuple_list:
        title_list.append(item[0])
    return title_list
    

def get_all_movie_info():
    select_q = """
                SELECT *
                FROM movies
                """
    cnx, c = connect_to_db()
    c.execute(q)
    results = c.fetchall()
    cnx.close()
    c.close()
    return results

def get_movie_names():
    name_q = """
            SELECT title
            FROM movies"""
    cnx, c = connect_to_db()
    c.execute(name_q)
    results = c.fetchall()
    cnx.close()
    c.close()
    title_list = convert_list_from_tuples(results) 
    return title_list

get_movie_names()
    

['Lock, Stock and Two Smoking Barrels',
 'The Kid',
 'Léon: The Professional',
 'What Ever Happened to Baby Jane?',
 'Taxi Driver',
 'The Hunt',
 'The Legend of 1900',
 'Perfect Blue',
 'Back to the Future',
 'The Hidden Fortress',
 'The Third Man',
 'Star Wars',
 'Song of the Sea',
 'Wolf Children',
 'Scarface',
 'Chungking Express',
 'Cinema Paradiso',
 'The Prestige',
 'Shutter Island',
 'Memories of Murder',
 'Ran',
 'The Best of Youth',
 'Sanjuro',
 'Yojimbo',
 'The Lord of the Rings: The Fellowship of the Ring',
 'The Lord of the Rings: The Two Towers',
 'The Lord of the Rings: The Return of the King',
 'Harry Potter and the Deathly Hallows: Part 2',
 'Grave of the Fireflies',
 'High and Low',
 'Princess Mononoke',
 'Spirited Away',
 'Forrest Gump',
 'Nostalgia',
 'Mirror',
 'Stalker',
 'Paperman',
 'The Departed',
 'Harakiri',
 'Ugetsu',
 'The 400 Blows',
 'The Tale of the Princess Kaguya',
 'Citizen Kane',
 'The Dark Knight',
 'Interstellar',
 'Miracle in Cell No. 7',
 "It's a 

In [13]:
######SQL QUERIES#########

def get_most_popular_movie():
    q = """SELECT * 
            FROM movies
            ORDER BY popularity DESC
            LIMIT 5
        """
    cnx, c = connect_to_db()
    c.execute(q)
    results = c.fetchall()
    cnx.close()
    c.close()
    return results

get_most_popular_movie()

def get_genre_counts():
    q = """SELECT genres.name AS genre_name, COUNT(*) 
        FROM genre_instances
        JOIN genres
        USING (genre_id)
        GROUP BY genre_instances.genre_id
    """
    cnx, c = connect_to_db()
    c.execute(q)
    results = c.fetchall()
    c.close()
    cnx.close()
    return results

get_genre_counts()

[('Music', 5),
 ('Romance', 30),
 ('Family', 13),
 ('War', 14),
 ('TV Movie', 1),
 ('Adventure', 23),
 ('Fantasy', 19),
 ('Animation', 21),
 ('Drama', 149),
 ('Horror', 6),
 ('Action', 27),
 ('Comedy', 42),
 ('History', 15),
 ('Western', 3),
 ('Thriller', 34),
 ('Crime', 32),
 ('Science Fiction', 17),
 ('Mystery', 20)]

In [20]:
def get_most_popular_genres():
    q = """
        SELECT genres.name, AVG(popularity) AS avg_popularity
        FROM genre_instances
        JOIN genres 
        USING (genre_id)
        JOIN movies
        USING (movie_id)
        GROUP BY genre_id
        ORDER BY avg_popularity DESC;
        """
    return execute_query(q)

get_most_popular_genres()

[('Science Fiction', 48.74058858086081),
 ('Adventure', 44.2043046536653),
 ('Action', 38.01070398754544),
 ('Fantasy', 22.458947589522914),
 ('Family', 22.250308000124416),
 ('Music', 22.177800369262695),
 ('Crime', 20.312937647104263),
 ('Thriller', 20.25147064994363),
 ('Western', 19.925333658854168),
 ('War', 18.92850008181163),
 ('Animation', 18.287238245918637),
 ('Drama', 16.203590624284423),
 ('Mystery', 16.20164997577667),
 ('Horror', 15.95549980799357),
 ('History', 15.361400032043457),
 ('Comedy', 14.026738007863363),
 ('Romance', 13.827366721630096),
 ('TV Movie', 6.376999855041504)]

In [ ]:
def insert_second_data_to_table()